In [1]:
!pip install pgmpy

    100% |████████████████████████████████| 296kB 7.5MB/s 
    100% |████████████████████████████████| 1.3MB 16.4MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
'''from google.colab import files
uploaded = files.upload()'''

'from google.colab import files\nuploaded = files.upload()'

In [2]:
! git clone https://github.com/kumarankit1996/Dataset.git

Cloning into 'Dataset'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 14180 (delta 7), reused 16 (delta 7), pack-reused 14164
Receiving objects: 100% (14180/14180), 184.13 MiB | 28.66 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Checking out files: 100% (42287/42287), done.


In [3]:
!ls Dataset/AML_Proj2/

 15features.csv			  feature_Unseen_weights_v1.h5
 31features_seen_train.csv	 'Icon'$'\r'
 31_Seen_Predict_Val.csv	  seen-dataset
 31_Seen_Train.csv		  seen_predict_feature.csv
 31_Seen_Val.csv		  seen_weights_v1_e10k.h5
 31_shuffled_Predict_Val.csv	  shuffled-dataset
 31_shuffled_Train.csv		  shuffled_predict_feature.csv
 31_shuffled_Val.csv		  shuffled_weights_v1_e10k.h5
 31_unSeen_Predict_Val.csv	  unseen-dataset
 31_unSeen_Train.csv		  unseen_predict_feature.csv
 31_unSeen_Val.csv		  Unseen_weights_v1_1.h5
 feature_seen_weights_v1.h5	  Unseen_weights_v1_e10k.h5
 feature_shuffled_weights_v1.h5


# Import Library

In [4]:
import numpy as np
import pandas as pd
import math
import io
from keras.layers import concatenate
import random
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Reshape, UpSampling2D,Flatten, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense
from keras import optimizers
import cv2
from PIL import Image
import os
from google.colab.patches import cv2_imshow
import math
from google.colab import files

Using TensorFlow backend.


# Import PGM library

In [0]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.estimators import K2Score
from pgmpy.inference import VariableElimination
from timeit import default_timer as timer
from pgmpy.estimators import ExhaustiveSearch
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BayesianEstimator

# Reading datasets

In [0]:
# read csv dataset
features=pd.read_csv("Dataset/AML_Proj2/15features.csv")
#removing unwanted columns
#features=features.drop(["imagename"],axis=1)
features.set_index('imagename',inplace=True)
all_nodes=['pen_pressure1', 'letter_spacing1', 'size1', 'dimension1', 'is_lowercase1',
       'is_continuous1', 'slantness1', 'tilt1', 'entry_stroke_a1', 'staff_of_a1',
       'formation_n1', 'staff_of_d1', 'exit_stroke_d1', 'word_formation1',
       'constancy1','pen_pressure2', 'letter_spacing2', 'size2', 'dimension2', 'is_lowercase2',
       'is_continuous2', 'slantness2', 'tilt2', 'entry_stroke_a2', 'staff_of_a2',
       'formation_n2', 'staff_of_d2', 'exit_stroke_d2', 'word_formation2',
       'constancy2','label']
feature_nodes=['pen_pressure1', 'letter_spacing1', 'size1', 'dimension1', 'is_lowercase1',
       'is_continuous1', 'slantness1', 'tilt1', 'entry_stroke_a1', 'staff_of_a1',
       'formation_n1', 'staff_of_d1', 'exit_stroke_d1', 'word_formation1',
       'constancy1','pen_pressure2', 'letter_spacing2', 'size2', 'dimension2', 'is_lowercase2',
       'is_continuous2', 'slantness2', 'tilt2', 'entry_stroke_a2', 'staff_of_a2',
       'formation_n2', 'staff_of_d2', 'exit_stroke_d2', 'word_formation2',
       'constancy2']

# seen predicted dataset
seen_predicted_features=pd.read_csv("Dataset/AML_Proj2/seen_predict_feature.csv")
seen_predicted_features.set_index('imagename',inplace=True)
# unseen predicted dataset
unseen_predicted_features=pd.read_csv("Dataset/AML_Proj2/unseen_predict_feature.csv")
unseen_predicted_features.set_index('imagename',inplace=True)
# seen predicted dataset
shuffled_predicted_features=pd.read_csv("Dataset/AML_Proj2/shuffled_predict_feature.csv")
shuffled_predicted_features.set_index('imagename',inplace=True)

In [17]:
seen_predicted_features

,imagename,pen_pressure,letter_spacing,size,dimension,is_lowercase,is_continuous,slantness,tilt,entry_stroke_a,staff_of_a,formation_n,staff_of_d,exit_stroke_d,word_formation,constancy
0,0683aa_num1.png,2,2,2,2,2,2,2,1,1,2,2,2,4,1,2
1,1154a_num1.png,2,1,1,1,2,2,1,1,1,1,1,3,2,1,1
2,1504aa_num1.png,2,2,2,2,2,1,2,1,1,2,2,3,2,2,2
3,1048a_num2.png,1,3,2,2,2,1,2,1,1,1,2,2,1,1,2
4,0063a_num2.png,2,1,1,1,2,2,3,1,1,2,2,2,3,1,1
5,0012a_num2.png,2,2,2,1,2,2,2,2,1,4,2,3,2,1,1
6,0097a_num2.png,2,3,3,3,2,1,2,1,1,2,2,3,1,1,2
7,1183a_num2.png,1,3,2,2,2,1,1,1,1,1,2,2,1,1,1
8,0481a_num2.png,2,3,2,2,2,2,1,1,1,2,1,2,4,1,1
9,0308a_num1.png,2,1,3,2,2,2,1,1,1,2,2,1,1,1,2


In [82]:
important=np.zeros((15,15))
# finding correlation of features
c=features.corr()
c

,pen_pressure,letter_spacing,size,dimension,is_lowercase,is_continuous,slantness,tilt,entry_stroke_a,staff_of_a,formation_n,staff_of_d,exit_stroke_d,word_formation,constancy
pen_pressure,1.000000,0.108567,-0.287659,-0.264762,0.091460,0.089430,0.095804,-0.023913,0.075880,-0.011692,-0.068007,0.099332,0.120479,-0.052080,-0.048135
letter_spacing,0.108567,1.000000,-0.051123,-0.059812,-0.027310,-0.123436,-0.034207,0.018311,-0.057954,0.034634,-0.009733,-0.035781,-0.005233,0.009613,0.006698
size,-0.287659,-0.051123,1.000000,0.694487,-0.085862,-0.273401,-0.182674,-0.023305,-0.133006,0.097090,0.293431,-0.170025,-0.230523,0.288011,0.251816
dimension,-0.264762,-0.059812,0.694487,1.000000,-0.092254,-0.259780,-0.173945,-0.011527,-0.118140,0.091562,0.291206,-0.157627,-0.200501,0.279895,0.246108
is_lowercase,0.091460,-0.027310,-0.085862,-0.092254,1.000000,0.154002,0.041868,0.012835,0.030322,0.144661,-0.041548,0.193493,0.123033,-0.040481,-0.011010
is_continuous,0.089430,-0.123436,-0.273401,-0.259780,0.154002,1.000000,0.099286,0.013101,0.137934,-0.040060,-0.211878,0.320322,0.306714,-0.175955,-0.144690
slantness,0.095804,-0.034207,-0.182674,-0.173945,0.041868,0.099286,1.000000,0.118365,0.060487,-0.037763,-0.054411,0.061823,0.040869,-0.087986,-0.034507
tilt,-0.023913,0.018311,-0.023305,-0.011527,0.012835,0.013101,0.118365,1.000000,0.013328,-0.007491,-0.032394,0.005806,-0.014586,-0.052607,-0.041449
entry_stroke_a,0.075880,-0.057954,-0.133006,-0.118140,0.030322,0.137934,0.060487,0.013328,1.000000,-0.022121,-0.012035,0.007808,0.077455,-0.009775,0.004379
staff_of_a,-0.011692,0.034634,0.097090,0.091562,0.144661,-0.040060,-0.037763,-0.007491,-0.022121,1.000000,0.111368,0.030424,0.004718,0.106431,0.121204


In [0]:
for i in range (0,15):
    for j in range(0,15):
        if c.iloc[i,j]<0.1:
            # if less than threshold then less correlation
            important[i][j]=0
        else:
            if i!=j:
              important[i][j]=1
df=pd.DataFrame.from_records(important,columns=feature_nodes[:15])
df

,pen_pressure1,letter_spacing1,size1,dimension1,is_lowercase1,is_continuous1,slantness1,tilt1,entry_stroke_a1,staff_of_a1,formation_n1,staff_of_d1,exit_stroke_d1,word_formation1,constancy1
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


# Task 2

# Bayesian Models

In [0]:
structure2=[('pen_pressure2','letter_spacing2'),('dimension2', 'size2'),('size2','slantness2'),('is_lowercase2', 'staff_of_a2'),('is_continuous2', 'exit_stroke_d2'),('is_continuous2', 'entry_stroke_a2'),('is_continuous2', 'is_lowercase2'),('slantness2', 'tilt2'),('formation_n2', 'constancy2'), ('formation_n2', 'word_formation2'), ('formation_n2', 'dimension2'),('formation_n2', 'size2'), ('formation_n2', 'staff_of_a2'),('staff_of_d2', 'is_continuous2'), ('staff_of_d2', 'exit_stroke_d2'), ('staff_of_d2', 'is_lowercase2'),('word_formation2', 'dimension2'), ('word_formation2', 'staff_of_a2'), ('word_formation2', 'size2'),('word_formation2', 'constancy2'), ('constancy2', 'staff_of_a2'),('constancy2', 'dimension2'),('constancy2', 'size2'),('pen_pressure2', 'exit_stroke_d2')]
structure1=[('pen_pressure1','letter_spacing1'),('dimension1', 'size1'),('size1','slantness1'),('is_lowercase1', 'staff_of_a1'),('is_continuous1', 'exit_stroke_d1'),('is_continuous1', 'entry_stroke_a1'),('is_continuous1', 'is_lowercase1'),('slantness1', 'tilt1'),('formation_n1', 'constancy1'), ('formation_n1', 'word_formation1'), ('formation_n1', 'dimension1'),('formation_n1', 'size1'), ('formation_n1', 'staff_of_a1'),('staff_of_d1', 'is_continuous1'), ('staff_of_d1', 'exit_stroke_d1'), ('staff_of_d1', 'is_lowercase1'),('word_formation1', 'dimension1'), ('word_formation1', 'staff_of_a1'), ('word_formation1', 'size1'),('word_formation1', 'constancy1'), ('constancy1', 'staff_of_a1'),('constancy1', 'dimension1'),('constancy1', 'size1'),('pen_pressure1', 'exit_stroke_d1')]

In [0]:
# find the k2 of the given data
#k2=K2Score(seen_train)
# Perform Hill Climb Search on the dataset
#hc=HillClimbSearch(features,scoring_method=k2)

In [0]:
# Predict the best model
#best_model=hc.estimate()
#print(best_model.edges())

In [0]:
def create_model(Train_data):
  model = BayesianModel([('is_lowercase1','is_lowercase_sim'),
                                ('is_lowercase2','is_lowercase_sim'),
                                ('is_continuous1','is_continuous_sim'),
                                ('is_continuous2','is_continuous_sim'),
                                ('is_lowercase_sim','is_continuous_sim'),
                                ('dimension1','dimension_sim'),
                                ('dimension2','dimension_sim'),
                                ('letter_spacing1','letter_spacing_sim'),
                                ('letter_spacing2','letter_spacing_sim'),
                                ('size1','size_sim'),
                                ('size2','size_sim'),
                                ('dimension_sim','size_sim'),
                                ('letter_spacing_sim','size_sim'),
                                ('constancy1','constancy_sim'),
                                ('constancy2','constancy_sim'),
                                ('size_sim','constancy_sim'),
                                ('word_formation1','word_formation_sim'),
                                ('word_formation2','word_formation_sim'),
                                ('constancy_sim','word_formation_sim'),
                                ('formation_n1','formation_n_sim'),
                                ('formation_n2','formation_n_sim'),
                                ('word_formation_sim','formation_n_sim'),
                                ('entry_stroke_a1','entry_stroke_a_sim'),
                                ('entry_stroke_a2', 'entry_stroke_a_sim'),
                         
                                ('pen_pressure1','pen_pressure_sim'),
                                ('pen_pressure2','pen_pressure_sim'),
                                ('entry_stroke_a_sim','entry_str_PP_sim'),
                                ('pen_pressure_sim','entry_str_PP_sim'),
                                ('entry_str_PP_sim','iscont_strPP_sim'),
                                ('is_continuous_sim','iscont_strPP_sim'),
                                ('iscont_strPP_sim','label'),
                                ('formation_n_sim','label')
                               ])
  # nx.draw(combined_model, **options)

  cpd_is_lowercase1 = TabularCPD('is_lowercase1',2,[[0.5],
                                                  [0.5]],
                                                  evidence=[], evidence_card=[])
  cpd_is_lowercase2 = TabularCPD('is_lowercase2',2,[[0.5],
                                                  [0.5]],
                                                  evidence=[], evidence_card=[])
  cpd_is_continuous1 = TabularCPD('is_continuous1',2,[[0.5],
                                                  [0.5]],
                                                  evidence=[], evidence_card=[])
  cpd_is_continuous2 = TabularCPD('is_continuous2',2,[[0.5],
                                                  [0.5]],
                                                  evidence=[], evidence_card=[])
  cpd_dimension1 = TabularCPD('dimension1',3,[[0.33],
                                      [0.34],[0.33]],
                                                  evidence=[], evidence_card=[])
  cpd_dimension2 = TabularCPD('dimension2',3,[[0.33],
                                      [0.34],[0.33]],
                                                  evidence=[], evidence_card=[])
  cpd_letter_spacing1 = TabularCPD('letter_spacing1',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])
  cpd_letter_spacing2 = TabularCPD('letter_spacing2',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])
  cpd_size1 = TabularCPD('size1',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])
  cpd_size2 = TabularCPD('size2',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])
  cpd_constancy1 = TabularCPD('constancy1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_constancy2 = TabularCPD('constancy2',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_word_formation1 = TabularCPD('word_formation1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_word_formation2 = TabularCPD('word_formation2',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_formation_n1 = TabularCPD('formation_n1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_formation_n2 = TabularCPD('formation_n2',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_entry_stroke_a1 = TabularCPD('entry_stroke_a1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_entry_stroke_a2 = TabularCPD('entry_stroke_a2',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_is_lowercase_sim = TabularCPD('is_lowercase_sim',2,[[0.1,0.9,0.9,0.1],
                                                              [0.9,0.1,0.1,0.9]],
                                                              evidence=['is_lowercase1','is_lowercase2'], 
                                                              evidence_card=[2,2])
  cpd_is_continuous_sim = TabularCPD('is_continuous_sim',2,[[0.9,0.1,0.9,0.6,0.9,0.6,0.9,0.1],
                                                              [0.1,0.9,0.1,0.4,0.1,0.4,0.1,0.9]],
                                                              evidence=['is_continuous1','is_continuous2','is_lowercase_sim'], 
                                                              evidence_card=[2,2,2])
  cpd_dimension_sim = TabularCPD('dimension_sim',2,[[0.1,0.8,0.9,0.8,0.1,0.8,0.9,0.8,0.1],
                                                  [0.9,0.2,0.1,0.2,0.9,0.2,0.1,0.2,0.9]],
                                                  evidence=['dimension1','dimension2'], evidence_card=[3,3])
  cpd_letter_spacing_sim = TabularCPD('letter_spacing_sim',2,[[0.1,0.8,0.9,0.8,0.1,0.8,0.9,0.8,0.1],
                                                  [0.9,0.2,0.1,0.2,0.9,0.2,0.1,0.2,0.9]],
                                                  evidence=['letter_spacing1','letter_spacing2'], evidence_card=[3,3])
  cpd_size_sim = TabularCPD('size_sim',2,[[0.6,0.3,0.3,0.1,0.8,0.7,0.7,0.3,0.9,0.8,0.7,0.4,0.7,0.6,0.6,0.3,0.6,0.3,0.3,0.1,0.8,0.4,0.4,0.85,0.9,0.8,0.8,0.3,0.8,0.4,0.4,0.85,0.6,0.3,0.3,0.1],
                                          [0.4,0.7,0.7,0.9,0.2,0.3,0.3,0.7,0.1,0.2,0.3,0.6,0.3,0.4,0.4,0.7,0.4,0.7,0.7,0.9,0.2,0.6,0.6,0.15,0.1,0.2,0.2,0.7,0.2,0.6,0.6,0.15,0.4,0.7,0.7,0.9]],
                                          evidence=['size1','size2','dimension_sim','letter_spacing_sim'], evidence_card=[3,3,2,2])
  cpd_constancy_sim = TabularCPD('constancy_sim',2,[[0.9,0.1,0.9,0.6,0.9,0.6,0.7,0.1],
                                          [0.1,0.9,0.1,0.4,0.1,0.4,0.3,0.9]],
                                          evidence=['constancy1','constancy2','size_sim'], evidence_card=[2,2,2])
  cpd_word_formation_sim = TabularCPD('word_formation_sim',2,[[0.9,0.1,0.9,0.7,0.9,0.7,0.9,0.1],
                                          [0.1,0.9,0.1,0.3,0.1,0.3,0.1,0.9]],
                                          evidence=['word_formation1','word_formation2','constancy_sim'], evidence_card=[2,2,2])
  cpd_formation_n_sim = TabularCPD('formation_n_sim',2,[[0.7,0.1,0.9,0.4,0.9,0.4,0.6,0.1],
                                          [0.3,0.9,0.1,0.6,0.1,0.6,0.4,0.9]],
                                          evidence=['formation_n1','formation_n2','word_formation_sim'], evidence_card=[2,2,2])
  cpd_entry_stroke_a_sim = TabularCPD('entry_stroke_a_sim',2,[[0.1,0.9,0.9,0.1],
                                                              [0.9,0.1,0.1,0.9]],
                                          evidence=['entry_stroke_a1','entry_stroke_a2'], evidence_card=[2,2])
  
  """cpd_exitStroke_d_1 = TabularCPD('exit_stroke_d',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])
  cpd_exitStroke_d_2 = TabularCPD('exit_stroke_d_1',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])

  cpd_staff_of_a_1 = TabularCPD('staff_of_a',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])
  cpd_staff_of_a_2 = TabularCPD('staff_of_a_1',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])

  cpd_staff_of_d_1 = TabularCPD('staff_of_d',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])
  cpd_staff_of_d_2 = TabularCPD('staff_of_d_1',3,[[0.33],
                                      [0.34],[0.33]],
                                      evidence=[], evidence_card=[])

  cpd_slantness_1 = TabularCPD('slantness',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])
  cpd_slantness_2 = TabularCPD('slantness_1',4,[[0.25],
                                      [0.25],[0.25],[0.25]],
                                      evidence=[], evidence_card=[])

  cpd_tilt1 = TabularCPD('tilt',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_tilt2 = TabularCPD('tilt_1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])"""

  cpd_pen_pressure1 = TabularCPD('pen_pressure1',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])
  cpd_pen_pressure2 = TabularCPD('pen_pressure2',2,[[0.5],
                                      [0.5]],
                                      evidence=[], evidence_card=[])

  cpd_pen_pressure_sim = TabularCPD('pen_pressure_sim',2,[[0.1,0.9,0.9,0.1],
                                                              [0.9,0.1,0.1,0.9]],
                                                              evidence=['pen_pressure1','pen_pressure2'], 
                                                              evidence_card=[2,2])

  cpd_entry_str_PP_sim=TabularCPD('entry_str_PP_sim',2,[[0.1,0.9,0.9,0.1],
                                                              [0.9,0.1,0.1,0.9]],
                                                              evidence=['entry_stroke_a_sim','pen_pressure_sim'], 
                                                              evidence_card=[2,2])

  cpd_iscont_strPP_sim=TabularCPD('iscont_strPP_sim',2,[[0.1,0.9,0.9,0.1],
                                                              [0.9,0.1,0.1,0.9]],
                                                              evidence=['entry_str_PP_sim','is_continuous_sim'], 
                                                              evidence_card=[2,2])

  cpd_label = TabularCPD('label',2,[[0.1,0.9,0.9,0.1],[0.9,0.1,0.1,0.9]],evidence=['iscont_strPP_sim','formation_n_sim'], evidence_card=[2,2])

  model.add_cpds(cpd_is_lowercase1,
                    cpd_is_lowercase2,
                    cpd_is_lowercase_sim,
                    cpd_is_continuous1,
                    cpd_is_continuous2,
                    cpd_is_continuous_sim,
                    cpd_dimension1,
                    cpd_dimension2,
                    cpd_dimension_sim,
                    cpd_letter_spacing1,
                    cpd_letter_spacing2,
                    cpd_letter_spacing_sim,
                    cpd_size1,
                    cpd_size2,
                    cpd_size_sim,
                    cpd_constancy1,
                    cpd_constancy2,
                    cpd_constancy_sim,
                    cpd_word_formation1,
                    cpd_word_formation2,
                    cpd_word_formation_sim,
                    cpd_formation_n1,
                    cpd_formation_n2,
                    cpd_formation_n_sim,
                    cpd_entry_stroke_a1,
                    cpd_entry_stroke_a2,


                    cpd_entry_stroke_a_sim,
                    cpd_pen_pressure1,
                    cpd_pen_pressure2,
                    cpd_pen_pressure_sim,
                    cpd_entry_str_PP_sim,
                    cpd_iscont_strPP_sim,
                    cpd_label
                   )
  model.check_model()
  '''
  
  # Create Bayesian Model
  model=BayesianModel()
  # add edges to bayesian model
  model.add_edges_from(structure1)
  # replicating structure
  model.add_edges_from(structure2)
  # adding validation node
  model.add_edges_from([('pen_pressure1','label'),('pen_pressure2','label'),('tilt1','label'),('tilt2','label'),
                        ('staff_of_d1','label'),('staff_of_d2','label'),('formation_n1','label'),('formation_n2','label'),
                        ('exit_stroke_d1','label'),('exit_stroke_d2','label'),('is_continuous1','label'),('is_continuous2','label')])
  #Train bayesian model on Train data
  model.fit(Train_data)'''

  return model

In [0]:
#print('K2 Score:'+str(k2.score(model)))

# Test the model

In [0]:
def predict(model,data):
  #data=data[:10]
  inference=VariableElimination(model)
  accu=0
  count_1 = 0 
  count_0 = 0
  counter=0
  features = {'letter_spacing1','size1','dimension1','is_lowercase1','is_continuous1','formation_n1','constancy1',#'pen_pressure1',
              'letter_spacing2','size2','dimension2','is_lowercase2','is_continuous2','formation_n2','constancy2'}#,'pen_pressure2'}

  var = {'letter_spacing_sim','size_sim','dimension_sim','is_lowercase_sim','is_continuous_sim','formation_n_sim','constancy_sim'}#,'pen_pressure_sim','entry_str_PP_sim','iscont_strPP_sim'}
  for index, row in data.iterrows():
    
    evidence = {}
    for f in features:
        evidence[f] = row[f] - 1
    infer = inference.query(variables=var, evidence=evidence)
    counter = counter + 1
    total = infer['letter_spacing_sim'].values[1]*100+ infer['size_sim'].values[1]*100+ infer['size_sim'].values[1]*100+ infer['is_lowercase_sim'].values[1]*100
    +infer['is_continuous_sim'].values[1]*100+infer['formation_n_sim'].values[1]*100+infer['constancy_sim'].values[1]*100#+ infer['pen_pressure_sim'].values[1]*100
    #+ infer['entry_str_PP_sim'].values[1]*100+ infer['iscont_strPP_sim'].values[1]*100
    total = total/7#10
    if(total > 50):
        label = 1
        count_1 += 1
    else:
        label = 0
        count_0 += 1
    if(label == row['label']):
        '''if(label == 1):
            count_1_1 += 1
        else:
            count_0_0 += 1'''
        accu += 1
  print('Validation Accuracy is: ' + str(round((accu/counter),3)*100))

# Concatinating dataset

In [0]:
def join_features(Train,name,feature):
  #loop for number of rows in dataset
  v=[]
  for i in range(0,len(Train.index)):
    try:
      #extract features for left
      x=feature.loc[Train.iloc[i,0],:]
      #extract features for right
      y=feature.loc[Train.iloc[i,1],:]
      #extract label
      w=Train.iloc[i,2]
      #concat all 3 side by side
      z=np.hstack([x,y,w])
      # add values to list
      v.append(z)
    except:
      continue
  print(len(v))
  # convert into dataframe
  v=pd.DataFrame(v)
  # rename columns to dataframe
  v.columns=all_nodes
  # write into csv
  v.to_csv(name+".csv",sep=",",index=False)
  files.download(name+".csv")

# SeenDataset

In [31]:
#read dataset
Train_seen_data=pd.read_csv("Dataset/AML_Proj2/seen-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)


#Train_seen_data.set_index('number',inplace=True)
#join_features(Train_seen_data,"31_Seen_Train")

#read dataset
Val_seen_data=pd.read_csv("Dataset/AML_Proj2/seen-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

#Val_seen_data.set_index('number',inplace=True)
#join_features(Val_seen_data,"31_Seen_Predict_Val",seen_predicted_features)



906


In [58]:
seen_train=pd.read_csv("Dataset/AML_Proj2/31_Seen_Train.csv")
seen_val=pd.read_csv("Dataset/AML_Proj2/31_Seen_Predict_Val.csv")
seen_val[:10]

,pen_pressure1,letter_spacing1,size1,dimension1,is_lowercase1,is_continuous1,slantness1,tilt1,entry_stroke_a1,staff_of_a1,...,slantness2,tilt2,entry_stroke_a2,staff_of_a2,formation_n2,staff_of_d2,exit_stroke_d2,word_formation2,constancy2,label
0,1,3,3,2,2,1,1,1,1,1,...,1,1,1,4,2,2,1,2,2,1
1,1,1,3,3,2,2,2,2,1,2,...,2,1,1,2,2,2,2,2,2,1
2,2,2,2,2,2,2,2,1,1,2,...,2,1,1,2,2,3,4,1,2,1
3,1,2,3,2,2,2,1,1,1,2,...,1,1,1,2,2,2,2,1,2,1
4,1,2,2,2,2,2,1,1,1,2,...,1,2,1,4,2,2,2,1,2,1
5,2,3,1,1,2,2,2,1,1,2,...,3,2,1,1,2,3,4,1,1,1
6,1,1,3,3,2,1,1,1,1,1,...,1,1,1,2,2,2,1,2,2,1
7,2,3,1,1,2,2,2,1,1,2,...,2,1,1,2,2,2,3,1,2,1
8,2,1,3,2,2,1,1,1,1,2,...,1,1,1,2,2,2,2,2,2,1
9,2,2,2,2,2,2,2,1,1,2,...,2,1,1,2,2,3,4,1,2,1


In [0]:
model=create_model(seen_train)

In [77]:
predict(model,seen_val)

Validation Accuracy is: 70.5


# Unseen Dataset

In [32]:
#read dataset
Train_unseen_data=pd.read_csv("Dataset/AML_Proj2/unseen-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)


#Train_unseen_data.set_index('number',inplace=True)
#join_features(Train_unseen_data,"31_unSeen_Train")

#read dataset
Val_unseen_data=pd.read_csv("Dataset/AML_Proj2/unseen-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

#Val_unseen_data.set_index('number',inplace=True)
#join_features(Val_unseen_data,"31_unSeen_Predict_Val",unseen_predicted_features)



7580


In [0]:
unseen_train=pd.read_csv("Dataset/AML_Proj2/31_unSeen_Train.csv")
unseen_val=pd.read_csv("Dataset/AML_Proj2/31_unSeen_Predict_Val.csv")


In [78]:
model=create_model(unseen_train)
predict(model,unseen_val)

Validation Accuracy is: 63.2


# Shuffled Dataset

In [33]:
#read dataset
Train_shuffled_data=pd.read_csv("Dataset/AML_Proj2/shuffled-dataset/dataset_seen_training_siamese.csv")
# drop the unwanted column
#Train_data=Train_data.drop("number",axis=1)
#Train_data=Train_data.sample(frac=1).reset_index(drop=True)

#Train_shuffled_data.set_index('number',inplace=True)
#join_features(Train_shuffled_data,"31_shuffled_Train")

#read dataset
Val_shuffled_data=pd.read_csv("Dataset/AML_Proj2/shuffled-dataset/dataset_seen_validation_siamese.csv")
# drop the unwanted column
#Val_data=Val_data.drop("number",axis=1)
#Val_data=Val_data.sample(frac=1).reset_index(drop=True)

Val_shuffled_data.set_index('number',inplace=True)
join_features(Val_shuffled_data,"31_shuffled_Predict_Val",shuffled_predicted_features)


5288


In [0]:
shuffled_train=pd.read_csv("Dataset/AML_Proj2/31_shuffled_Train.csv")
shuffled_val=pd.read_csv("Dataset/AML_Proj2/31_shuffled_Predict_Val.csv")


In [84]:
model=create_model(shuffled_train)
predict(model,shuffled_val)

Validation Accuracy is: 57.2
